# Russian Synodal Bible (1885)

## Import

### Libraries

In [1]:
import os
from itertools import chain
from lxml import etree
from backend import *
import matplotlib
import string

### Definitions

In [2]:
bibleTXT = '../texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './proc/booksDict.json'
bibleJSON = '../texts/bible/bible.json'
bibleIdJSON = '../texts/bible/bibleID.json'
bibleXML = '../texts/bible/bible.xml'
bibFrazyTXT = '../texts/bible/Dubrovina_Slovar_Bibleyskikh_Frazeologizmov.txt'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [3]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

In [4]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,\n
1,== Бытие ==\n
2,\n
3,=== 1 ===\n
4,\n
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Tidy Up

In [5]:
# remove blank lines and select puncutation, keeping only header & clean text lines
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n|^\s$', '')

#unwanted_parts = bibliiaDf.loc[bibliiaDf.text.str.contains('Предисловие|Примечания')].index.tolist() + bibliiaDf.loc[bibliiaDf.text.str.contains('^[^\d|^=]')].index.tolist() + [x for x in range(21483,21490)] + [x for x in range(16571,16577)]

# concat list of unwanted sections' indices
get_rid = chain(range(13259,13261), range(15172,15175), range(16571,16578), range(21483,21491))

# drop notes section from Book of Job
bibliiaDf = bibliiaDf.drop(index=[x for x in get_rid])

# rename foreword section for Esther and Wisdom of Jesus to match chapter style
#bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('=== Предисловие ===', '=== 0 ===')

# remove various errata and notes marks
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '']#.reset_index().drop(columns="index")

bibliiaDf

/tmp/ipykernel_37423/2067043555.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n|^\s$', '')
/tmp/ipykernel_37423/2067043555.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')


,text
1,== Бытие ==
3,=== 1 ===
5,1 В начале сотворил Бог небо и землю.
6,"2 Земля же была безвидна и пуста, и тьма над б..."
7,3 И сказал Бог: да будет свет. И стал свет.
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


In [6]:
#bibliiaDf.loc[13259:13260] # MOLITVA and gospodi vsederzhiteliu
#bibliiaDf.loc[15172:15174] # predislovie for Esfira
#bibliiaDf.loc[16571:16577] # notes for Job
#bibliiaDf.loc[21483:21490] # предисловие for WisdJes

### Find Parts

In [7]:
# find df rows containing all the book and chapter title lines 
# as well as the OT/NT split on the 1st page of Matthew
chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index
test_line = bibliiaDf.loc[bibliiaDf.text.str.contains(matt_name)].index

# combine the title lines for extraction later
title_lines = test_line.append([chap_lines, book_lines]).sort_values()

/tmp/ipykernel_217180/2439614081.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
/tmp/ipykernel_217180/2439614081.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index


### Assign OHCO Labels

In [8]:
# ['test']
bibliiaDf[bibleOHCO[0]] = np.where(bibliiaDf.index<test_line[0], 1, 2)
# ['book']
bibliiaDf[bibleOHCO[1]] = bibliiaDf.loc[book_lines].text.str.extract(book_lines_re)
# ['chap']
bibliiaDf[bibleOHCO[2]] = bibliiaDf.loc[chap_lines].text.str.extract(chap_lines_re)
# ['verse'] by splitting verse num at beginning from text in current verse lines
bibliiaDf[[bibleOHCO[3],'text']] = bibliiaDf.text.str.split(' ', 1, expand=True).iloc[:, [0, 1]]

# fill in book and chapter titles to cells below them
bibliiaDf[bibleOHCO[1:3]] = bibliiaDf[['book','chap']].ffill()

# transfer book numbers onto book titles
#bibliiaDf[bibleOHCO[1]] = bibliiaDf[bibleOHCO[1]].map(dict([(y,x) for (x,y) in enumerate(bibliiaDf.book.unique(), start=1)]))

# drop title lines and reset index to give verse id num 
bibliiaDf = bibliiaDf.drop(title_lines, axis=0).reset_index()#.set_index(bibleOHCO)
bibliiaDf.index = range(1,len(bibliiaDf)+1)
bibliiaDf.index.name = 'v_id'

# reorder columns
bibliiaDf = bibliiaDf[['test', 'book', 'chap', 'verse', 'text']]
bibliiaDf

,test,book,chap,verse,text
v_id,,,,,
1,1,Бытие,1,1,В начале сотворил Бог небо и землю.
2,1,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,1,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.
4,1,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,1,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37082,2,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37083,2,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...
37084,2,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...


### Swap Russian Book Name for English Equivalent

In [9]:
booksDict

{'1': {'ru_name': 'Книга Бытия',
  'en_name': 'Genesis',
  'ru_abbr': 'Быт',
  'en_abbr': 'Gen',
  'chap_verse': {'1': 31,
   '2': 25,
   '3': 24,
   '4': 26,
   '5': 32,
   '6': 22,
   '7': 24,
   '8': 22,
   '9': 29,
   '10': 32,
   '11': 32,
   '12': 20,
   '13': 18,
   '14': 24,
   '15': 21,
   '16': 16,
   '17': 27,
   '18': 33,
   '19': 38,
   '20': 18,
   '21': 34,
   '22': 24,
   '23': 20,
   '24': 67,
   '25': 34,
   '26': 35,
   '27': 46,
   '28': 22,
   '29': 35,
   '30': 43,
   '31': 55,
   '32': 32,
   '33': 20,
   '34': 31,
   '35': 29,
   '36': 43,
   '37': 36,
   '38': 30,
   '39': 23,
   '40': 23,
   '41': 57,
   '42': 38,
   '43': 34,
   '44': 34,
   '45': 28,
   '46': 34,
   '47': 31,
   '48': 22,
   '49': 33,
   '50': 26}},
 '2': {'ru_name': 'Книга Исход',
  'en_name': 'Exodus',
  'ru_abbr': 'Исх',
  'en_abbr': 'Ex',
  'chap_verse': {'1': 22,
   '2': 25,
   '3': 22,
   '4': 31,
   '5': 23,
   '6': 30,
   '7': 25,
   '8': 32,
   '9': 35,
   '10': 29,
   '11': 10,
   

In [10]:
book_nums = [(x+1,y) for x,y in enumerate(bibliiaDf.book.unique())]
bibliiaDf['book_en'] = [booksDict[str(x)]['en_name'] for x,y in book_nums for name in bibliiaDf.book if y == name]

In [11]:
bibliiaDf

,test,book,chap,verse,text,book_en
v_id,,,,,,
1,1,Бытие,1,1,В начале сотворил Бог небо и землю.,Genesis
2,1,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без...",Genesis
3,1,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.,Genesis
4,1,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог...",Genesis
5,1,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве...",Genesis
...,...,...,...,...,...,...
37082,2,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...,Book of Revelation
37083,2,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...,Book of Revelation
37084,2,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...,Book of Revelation


### Export to JSON

In [12]:
bibliiaDf.to_json(bibleJSON, orient='index')

## Processing

In [13]:
#bibliiaDf = pd.read_json(bibleJSON, orient='index')
#bibliiaDf.index.name = 'v_id'
BibTextDf = bibliiaDf[['text']]
BibLibDf = bibliiaDf[bibleOHCO]
bibliiaDf

,test,book,chap,verse,text,book_en
v_id,,,,,,
1,1,Бытие,1,1,В начале сотворил Бог небо и землю.,Genesis
2,1,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без...",Genesis
3,1,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.,Genesis
4,1,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог...",Genesis
5,1,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве...",Genesis
...,...,...,...,...,...,...
37082,2,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...,Book of Revelation
37083,2,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...,Book of Revelation
37084,2,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...,Book of Revelation


### OHCO DFs

#### Dictionaries for testament and book IDs

In [14]:
#testsDict = dict(enumerate(bibliiaDf.test.unique()))
testsDict = dict([(value, key) for key, value in dict(enumerate(bibliiaDf.test.unique())).items()])
booksDict = dict([(value, key) for key, value in dict(enumerate(bibliiaDf.book.unique())).items()])

#### Testaments

In [15]:
# make df of testament texts

TestsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.\
    groupby('test', sort=False).text], columns=['test', 'text']).\
        set_index(np.arange(1,len(bibliiaDf.test.unique())+1))
TestsDf.index.name = 't_id'
TestsDf 

,test,text
t_id,,
1,1,В начале сотворил Бог небо и землю. Земля же б...
2,2,"Родословие Иисуса Христа, Сына Давидова, Сына ..."


#### Books

In [16]:
## make Df of book texts

BooksDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.\
    groupby('book', sort=False).text], columns=['book', 'text'])\
    .set_index(np.arange(1,len(bibliiaDf.book.unique())+1))
BooksDf.index.name = 'b_id'

BooksDf 

,book,text
b_id,,
1,Бытие,В начале сотворил Бог небо и землю. Земля же б...
2,Исход,"Вот имена сынов Израилевых, которые вошли в Е..."
3,Левит,И воззвал Господь к Моисею и сказал ему из ски...
4,Числа,"И сказал Господь Моисею в пустыне Синайской, в..."
5,Второзаконие,"Сии суть слова, которые говорил Моисей всем Из..."
...,...,...
73,Второе послание к Тимофею святого апостола Павла,"Павел, волею Божиею Апостол Иисуса Христа, по ..."
74,Послание к Титу святого апостола Павла,"Павел, раб Божий, Апостол же Иисуса Христа, по..."
75,Послание к Филимону святого апостола Павла,"Павел, узник Иисуса Христа, и Тимофей брат, Фи..."


#### Chapters

In [17]:
# grab book name and chapter number from main df
ChapsDf = pd.DataFrame([(x, ' '.join(y)) for (x,y) in bibliiaDf.groupby(bibleOHCO[1:3], sort=False).text], columns=[('book', 'chap'), 'text'])
ChapsDf[['book','chap']] = pd.DataFrame(list(ChapsDf[('book', 'chap')]), index=ChapsDf.index, columns=bibleOHCO[1:3])
del ChapsDf[('book', 'chap')]
#ChapsDf = ChapsDf.replace({"book": booksDict}).sort_values(by=bibleOHCO[1:3], ascending=[True, True])

ChapsDf = ChapsDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(ChapsDf)+1))
ChapsDf.index.name = 'c_id'

#ChapsDf = ChapsDf.replace({"book":dict(enumerate(bibliiaDf.book.unique()))})
#ChapsDf = ChapsDf.reset_index().set_index(bibleOHCO[:3])

ChapsDf

,text,book,chap
c_id,,,
1,В начале сотворил Бог небо и землю. Земля же б...,Бытие,1
2,Так совершены небо и земля и все воинство их. ...,Бытие,2
3,"Змей был хитрее всех зверей полевых, которых с...",Бытие,3
4,"Адам познал Еву, жену свою; и она зачала, и ро...",Бытие,4
5,Вот родословие Адама: когда Бог сотворил челов...,Бытие,5
...,...,...,...
1357,"После сего я увидел иного Ангела, сходящего с ...",Откровение святого Иоанна Богослова,18
1358,После сего я услышал на небе громкий голос как...,Откровение святого Иоанна Богослова,19
1359,"И увидел я Ангела, сходящего с неба, который и...",Откровение святого Иоанна Богослова,20


In [18]:
# make t_id and b_id columns with TestsDf and BooksDf IDs respectively
bibliiaDf['t_id'] = bibliiaDf['test'].map(TestsDf.reset_index().set_index('test')['t_id'])
bibliiaDf['b_id'] = bibliiaDf['book'].map(BooksDf.reset_index().set_index('book')['b_id'])

# merge existing bibliiaDf with Chapsdf for c_id column (because it's based on multiple matching columns)
bibliiaDf = pd.merge(bibliiaDf, ChapsDf.reset_index()[['book', 'chap', 'c_id']], on=['book', 'chap'], how='left')

# remake index
bibliiaDf = bibliiaDf.reset_index().drop(['index'], axis=1).set_index(np.arange(1,len(bibliiaDf)+1))
bibliiaDf.index.name = 'v_id'

bibliiaDf

,test,book,chap,verse,text,book_en,t_id,b_id,c_id
v_id,,,,,,,,,
1,1,Бытие,1,1,В начале сотворил Бог небо и землю.,Genesis,1,1,1
2,1,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без...",Genesis,1,1,1
3,1,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.,Genesis,1,1,1
4,1,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог...",Genesis,1,1,1
5,1,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве...",Genesis,1,1,1
...,...,...,...,...,...,...,...,...,...
37082,2,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...,Book of Revelation,2,77,1361
37083,2,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...,Book of Revelation,2,77,1361
37084,2,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...,Book of Revelation,2,77,1361


In [42]:
bibliiaDf.to_json(bibleIdJSON, orient='index')

### Make XML

In [21]:
bibliiaDf = pd.read_json(bibleIdJSON, orient='index')

In [19]:
bibliiaDf#.rename(columns={'book':'book_ru'})

,test,book,chap,verse,text,book_en,t_id,b_id,c_id
v_id,,,,,,,,,
1,1,Бытие,1,1,В начале сотворил Бог небо и землю.,Genesis,1,1,1
2,1,Бытие,1,2,"Земля же была безвидна и пуста, и тьма над без...",Genesis,1,1,1
3,1,Бытие,1,3,И сказал Бог: да будет свет. И стал свет.,Genesis,1,1,1
4,1,Бытие,1,4,"И увидел Бог свет, что он хорош, и отделил Бог...",Genesis,1,1,1
5,1,Бытие,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве...",Genesis,1,1,1
...,...,...,...,...,...,...,...,...,...
37082,2,Откровение святого Иоанна Богослова,22,17,И Дух и невеста говорят: прииди! И слышавший д...,Book of Revelation,2,77,1361
37083,2,Откровение святого Иоанна Богослова,22,18,И я также свидетельствую всякому слышащему сло...,Book of Revelation,2,77,1361
37084,2,Откровение святого Иоанна Богослова,22,19,и если кто отнимет что от слов книги пророчест...,Book of Revelation,2,77,1361


In [20]:
bookRefDict = {}
for book in booksDict.items():
    cleanDict = {int(book[0]):book[1]}
    cleanDict[int(book[0])]['chap_verse'] = bibliiaDf.loc[bibliiaDf.b_id == int(book[0])].groupby('chap').max().to_dict()['verse']
    #print(verseDict)
    #cleanDict.update(verseDict)
    #print(cleanDict)
    bookRefDict.update(cleanDict)
bookRefDict

ValueError: invalid literal for int() with base 10: 'Бытие'

In [21]:
with open('../site/data/booksDict.json', "w") as outfile:
    json.dump(bookRefDict, outfile)
    #json.dump('../site/data/booksDict.json', outfile)

In [30]:
bibliiaDf.loc[bibliiaDf.book=='Бытие'].book_en.unique()[0]

'Genesis'

In [31]:
root = etree.Element("bible")
print(root.tag)
t = b = c = v = 1
#TestList = [x for x in bibliiaDf.test.unique()]
#for test in range(len(TestList)): 
#    root.append(etree.Element("t", n=str(t), name=TestList[test]))
BookList = list(bibliiaDf.book.unique())
for book in range(len(BookList)): 
    root.append(etree.Element("b", n=str(b), name_ru=BookList[book], name_en=bibliiaDf.loc[bibliiaDf.book==BookList[book]].book_en.unique()[0]))
    ChapList = list(bibliiaDf.loc[bibliiaDf['book'] == BookList[book]].chap.unique())
    for chap in range(len(ChapList)): 
        root[book].append(etree.Element("c", n=str(c), name=str(chap+1)))
        VerseList = list(bibliiaDf.loc[(bibliiaDf['book'] == BookList[book]) & (bibliiaDf['chap'] == ChapList[chap])].index)
        for verse in range(len(VerseList)): 
            root[book][chap].append(etree.Element("v", n=str(v), name=str(verse+1)))
            verse_text = bibliiaDf.loc[v].text
            root[book][chap][verse].text = verse_text
            v+=1
        c+=1
    b+=1
#t+=1
#print(etree.tostring(root, pretty_print=True, xml_declaration=True))
etree.ElementTree(root).write(bibleXML, pretty_print=True, xml_declaration=True, encoding='windows-1251')

bible


## Token DFs

In [ ]:
%%time
VerseTokenDf = nat_parse(BibTextDf)

In [ ]:
TokenDf = pd.read_pickle('./proc/BibleVerseTokenDf.pkl')

# add stopword boolean, True if all attribute columns are null
TokenDf['stopword'] = ~TokenDf.loc[:, tokenCols[9:]].any(axis=1)
TokenDfIdx = pd.Index(range(1, (TokenDf.shape[0]+1)), name='id')
TokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

TokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,В,1_1,1_2,case,ADP,в,None,...,None,None,None,None,None,None,None,None,None,True
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676262,37110,32,38,Христа,1_5,1_4,flat:name,PROPN,христос,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
676263,37110,39,41,со,1_6,1_8,case,ADP,с,None,...,None,None,None,None,None,None,None,None,None,True
676264,37110,42,47,всеми,1_7,1_8,det,DET,весь,None,...,Ins,None,None,None,Plur,None,None,None,None,False


In [ ]:
TokenDf.loc[TokenDf['text'] == '=']

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
282187,13735,12,13,=,1_2,1_0,root,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
282188,13735,13,14,=,1_3,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
282189,13735,14,15,=,1_4,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
301956,14974,11,12,=,1_2,1_0,root,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
301957,14974,12,13,=,1_3,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
301958,14974,13,14,=,1_4,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
356584,19208,12,13,=,1_2,1_0,root,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
356585,19208,13,14,=,1_3,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True
356586,19208,14,15,=,1_4,1_2,punct,SYM,=,None,...,None,None,None,None,None,None,None,None,None,True


In [ ]:
# stopword counts

TokenDf.loc[TokenDf.stopword == True].lemma.value_counts()[:25]

и        57443
в        18350
не       11883
на        8968
с         6184
от        5042
к         4863
что       4792
из        4544
как       3659
а         3565
по        3089
ибо       2934
но        2905
же        2831
у         2714
за        2455
о         2444
для       2389
когда     2203
если      1851
вот       1731
то        1651
чтобы     1548
его       1486
Name: lemma, dtype: int64

In [ ]:
TokenDf.loc[TokenDf[tokenCols[9]] == 'Anim'].lemma.value_counts().to_frame()

,lemma
господь,6865
сын,4698
бог,4330
царь,3216
человек,2310
...,...
гацорфия,1
цалафа,1
конский,1
фекоец,1


In [ ]:
GetRankDf(TokenDf, 'lemma')#.plot(x='rank')

,n,rank
lemma,,
он,19656,1
они,12325,2
быть,10839,3
я,10762,4
ты,7851,5
...,...,...
таинница,1,20821
избирательница,1,20822
добивать,1,20823


In [ ]:
GetRankDf(TokenDf, 'pos')

,n,rank
pos,,
NOUN,147026,1
VERB,108063,2
PRON,78480,3
PROPN,55273,4
ADJ,35325,5
DET,31955,6
ADV,21293,7
AUX,10141,8
NUM,7032,9


In [ ]:
GetRankDf(TokenDf, 'rel')

,n,rank
rel,,
nsubj,63632,1
conj,61236,2
obl,56304,3
obj,53076,4
nmod,43336,5
root,42277,6
det,25652,7
amod,25529,8
iobj,17714,9


In [ ]:
GetRankDf(TokenDf, 'anim')

,n,rank
anim,,
Inan,145289,1
Anim,83455,2


In [ ]:
GetRankDf(TokenDf, 'person')

,n,rank
person,,
3,57935,1
2,22086,2
1,19806,3


In [ ]:
VocabDf = TokenDf.loc[TokenDf.stopword != True].lemma.value_counts().to_frame().reset_index()
VocabDf.columns = ['lemma', 'count']
VocabDf.index = VocabDf.index.map(lambda x: x + 1).rename('rank')
VocabDf.iloc[:25]#.plot(x='rank')

,lemma,count
rank,,
1,он,19656
2,они,12325
3,быть,10839
4,я,10762
5,ты,7851
6,господь,7101
7,весь,6982
8,свой,6038
9,твой,5958


In [ ]:
bibliiaType = {'test':str, 'book':str, 'chap':int, 'verse':int}
bibliiaDf = pd.read_json(bibleJSON, orient='index', dtype=bibliiaType)
bibliiaDf.index.name = 'p_id'
#bibliiaDf.verse = bibliiaDf.verse.astype(int)
p_idDict = bibliiaDf.drop('text', axis=1).to_dict(orient='index')
bibliiaDf

,test,book,chap,verse,text
p_id,,,,,
1,O,Gen,1,1,В начале сотворил Бог небо и землю.
2,O,Gen,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,O,Gen,1,3,И сказал Бог: да будет свет. И стал свет.
4,O,Gen,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,O,Gen,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37099,N,Rev,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37100,N,Rev,22,18,И я также свидетельствую всякому слышащему сло...
37101,N,Rev,22,19,и если кто отнимет что от слов книги пророчест...


In [ ]:
# dictionary with verse lines in format 
# {book: {0: first_line, last_line}, {chap_num: (first_chap_line, last_chap_line)}}
PartLineNumsDict = {}
for test_name in bibliiaDf.test.unique():
    t_min = bibliiaDf.loc[bibliiaDf['test'] == test_name].reset_index().p_id.min()
    t_max = bibliiaDf.loc[bibliiaDf['test'] == test_name].reset_index().p_id.max()
    print(f"{test_name}: {t_min} - {t_max}")
    for book_name in bibliiaDf.book.unique():
        b_min = bibliiaDf.loc[bibliiaDf['book'] == book_name].reset_index().p_id.min()
        b_max = bibliiaDf.loc[bibliiaDf['book'] == book_name].reset_index().p_id.max()
        PartLineNumsDict.update({book_name: {0: (b_min, b_max)}})
        #print(f"{book_name}: {b_min} - {b_max}")
        for chap_name in bibliiaDf.loc[bibliiaDf['book'] == book_name].chap.unique():
            c_min = bibliiaDf.loc[(bibliiaDf['book'] == book_name) & (bibliiaDf['chap'] == chap_name)].reset_index().p_id.min()
            c_max = bibliiaDf.loc[(bibliiaDf['book'] == book_name) & (bibliiaDf['chap'] == chap_name)].reset_index().p_id.max()
            PartLineNumsDict[book_name][chap_name] = (c_min, c_max)
            #    print(f"{chap_name}: {c_min} - {c_max}")
#print(PartLineNumsDict)

O: 1 - 29140


KeyboardInterrupt: 

In [ ]:

TokenDf.loc[(TokenDf['p_id'] >= count_start) & (TokenDf['p_id'] <= count_stop)].lemma.value_counts()

и                 57470
он                19656
в                 18350
они               12325
не                11883
                  ...  
выспросить            1
рассудительные        1
воздаст               1
скоротечный           1
закупать              1
Name: lemma, Length: 20984, dtype: int64

In [ ]:
[x for x in PartLineNumsDict]

['Gen',
 'Ex',
 'Lev',
 'Num',
 'Deut',
 'Josh',
 'Judg',
 'Ruth',
 '1Sam',
 '2Sam',
 '1Kings',
 '2Kings',
 '1Chron',
 '2Chron',
 '1Ezra',
 'Neh',
 '2Ezra',
 'Tob',
 'Jud',
 'Esther',
 'Job',
 'Ps',
 'Prov',
 'Eccles',
 'Song',
 'WisdSol',
 'WisdJus',
 'Is',
 'Jer',
 'Lam',
 'Bar',
 'Ezek',
 'Dan',
 'Hos',
 'Joel',
 'Amos',
 'Obad',
 'Jon',
 'Mic',
 'Nahum',
 'Hab',
 'Zeph',
 'Hag',
 'Zech',
 'Mal',
 '1Mac',
 '2Mac',
 '3Mac',
 '3Esd',
 'Mt',
 'Mk',
 'Lk',
 'Jn',
 'Acts',
 'Jas',
 '1Pet',
 '2Pet',
 '1Jn',
 '2Jn',
 '3Jn',
 'Jude',
 'Rom',
 '1 Cor',
 '2 Cor',
 'Gal',
 'Eph',
 'Phil',
 'Col',
 '1Thess',
 '2Thess',
 '1Tim',
 '2Tim',
 'Tit',
 'Philem',
 'Heb',
 'Rev']

### Manual TF-IDF

In [ ]:
# TF-IDF

TfidfDf = TokenDf.reindex(columns=['text', 'lemma', 'tfidf_bible', 'tfidf_book', 'tfidf_chap'])
#vocab = TokenDf.loc[TokenDf['p_id'] <= 10, 'lemma'].unique().tolist()

def getTFIDF(count_start=0, count_stop=(TokenDf.shape[0]+1)):

    return TFIDF

count_start = 0
count_stop = (TokenDf.shape[0]+1)

TfidfCountDf = TokenDf.loc[(TokenDf['p_id'] >= count_start) & (TokenDf['p_id'] <= count_stop)][['p_id', 'lemma']]
VocabCountDict = TfidfCountDf.value_counts('lemma').to_dict()

total_words = sum(VocabCountDict.values())
num_docs = TfidfCountDf.p_id.unique().size
    
    for lemma in VocabCountDict.keys(): 

    instances = VocabCountDict.get(lemma)
    docsw_word = TfidfCountDf.loc[TfidfCountDf['lemma'] == lemma].p_id.unique().size

    TF = instances / total_words
    IDF = num_docs / docsw_word
    TFIDF = TF * IDF

    TfidfDf['tfidf_bible'].loc[TfidfDf['lemma'] == lemma] = TFIDF

#TfidfDf['tfidf_bible'] = TokenDf

# PartLineNumsDict = {book: {0: first_line, last_line}, {chap_num: (first_chap_line, last_chap_line)}}

TfidfDf

/home/xtra/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1714: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,text,lemma,tfidf_bible,tfidf_book,tfidf_chap
id,,,,,
1,В,в,0.075158,NaN,NaN
2,начале,начало,0.055755,NaN,NaN
3,сотворил,сотворить,0.057942,NaN,NaN
4,Бог,бог,0.063870,NaN,NaN
5,небо,небо,0.058612,NaN,NaN
...,...,...,...,...,...
676262,Христа,христос,0.058178,NaN,NaN
676263,со,с,0.066290,NaN,NaN
676264,всеми,весь,0.067202,NaN,NaN


In [ ]:
TfidfDf.to_pickle('./proc/TfidfDf.pkl')

In [ ]:
TfidfDf.sort_values('tfidf_bible', ascending=False).lemma.unique()[:20]

array(['глядели', 'скопец', 'любодейный', 'воспрянь', '=', 'окропили',
       'элнафан', 'трехлетний', 'отвергаться', 'хусарсафема', 'и',
       'связка', 'кивот', 'кладязь', 'восполнение', 'обрызгать', 'саруий',
       'государям', 'силомский', 'вкупе'], dtype=object)

## NER-supplemented Bible text

pronominal references swapped with names

## Bible Phrase Dictionary

In [ ]:
with open(bibFrazyTXT) as bibleFrazy: 
    bibleFrazyLines = bibleFrazy.readlines()

In [ ]:
BibleFrazyDf = pd.DataFrame(bibleFrazyLines).rename(columns={0:'fraza'})
BibleFrazyDf.loc[:,'fraza'] = BibleFrazyDf.loc[:,'fraza'].apply(lambda x: x.strip().strip('\n').translate(str.maketrans('', '', string.punctuation)))
BibleFrazyDf = BibleFrazyDf.loc[BibleFrazyDf.fraza != '']

In [ ]:
frazyList = BibleFrazyDf.loc[30189:].fraza.to_list()
FrazyDf = pd.DataFrame(data=frazyList).rename(columns={0:'fraza'})
FrazyDf.index.name = 'f_id'
FrazyDf

,fraza
f_id,
0,Агнец Божий непорочный как будто как будто сло...
1,Адамова голова
2,Адамово яблоко
3,Адамовы веки времена
4,Адамовы слёзы
...,...
535,Что посеешь то и пожнёшь
536,Юдоль плача печали скорби бед плачевная земная...
537,Явление Христа народу


In [ ]:
%%capture
FrazyDf = nat_parse(FrazyDf, textCol='fraza')

In [ ]:
FrazyDf['stopword'] = ~FrazyDf.loc[:, tokenCols[9:]].any(axis=1)
#FrazyDf = FrazyDf.set_index('p_id', 'token_id')
FrazyDf

/tmp/ipykernel_103847/3221199829.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only
  FrazyDf = FrazyDf.set_index('p_id', 'token_id')


,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
p_id,,,,,,,,,,,,,,,,,,,,
0,0,5,Агнец,1_1,1_3,nsubj,PROPN,агнец,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None,False
0,6,11,Божий,1_2,1_1,det,ADJ,божий,None,None,Nom,Pos,Masc,None,Sing,None,None,None,None,False
0,12,22,непорочный,1_3,1_0,root,VERB,непорочный,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Mid,False
0,23,26,как,1_4,1_10,mark,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,True
0,27,32,будто,1_5,1_6,fixed,PART,будто,None,None,None,None,None,None,None,None,None,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,4,7,аки,1_2,1_1,obj,NOUN,аки,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None,False
539,8,11,как,1_3,1_4,case,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,True
539,12,16,тать,1_4,1_2,acl,NOUN,тать,Inan,None,Nom,None,Masc,None,Sing,None,None,None,None,False


In [ ]:
for phrase_id in FrazyDf.p_id.unique().tolist():
   testPhrase = FrazyDf.loc[FrazyDf.p_id] 

In [ ]:
pos_lists = []
for x in range(0,(FrazyDf.p_id.max()+1)):
    pos_lists.append(FrazyDf.loc[(FrazyDf.p_id == x) & (FrazyDf.stopword == False)].lemma.unique())

AttributeError: 'DataFrame' object has no attribute 'p_id'

In [ ]:
#bibliiaDf.loc[(bibliiaDf.test == 'N')]
bibliiaDf.loc[(bibliiaDf.book == 'Lk') & (bibliiaDf.chap == 10) & (bibliiaDf.verse == '42')].index

Int64Index([31412], dtype='int64', name='p_id')

In [ ]:
bibliiaDf.loc[31412].text

'а одно только нужно; Мария же избрала благую часть, которая не отнимется у нее. '

In [ ]:
#phraseList = [x.tolist() for x in pos_lists]
for verse_id in range(31400,31450):
    print(f"verse_id: {verse_id}")
    testVerse = TokenDf.loc[(TokenDf.p_id == verse_id) & (TokenDf.stopword == False)].lemma.tolist()
    print(f"testVerse: {testVerse}")
    for phrase in pos_lists:
        print(f"phrase: {phrase}")
        i = 0
        for word in phrase:
            if word in testVerse:
                i+=1
        print(f"{verse_id} : {i} / {phrase.size}")

verse_id: 31400
testVerse: ['на', 'это', 'сказать', 'иисус', 'некоторый', 'человек', 'идти', 'из', 'иерусалим', 'в', 'иерихон', 'и', 'попасться', 'разбойник', 'который', 'снять', 'с', 'он', 'одежда', 'изранить', 'он', 'и', 'уйти', 'оставить', 'он', 'едва', 'живой']
phrase: ['агнец' 'божий' 'непорочный' 'как' 'будто' 'словно' 'точно']
31400 : 0 / 7
phrase: ['адамов' 'голова']
31400 : 0 / 2
phrase: ['адамов' 'яблоко']
31400 : 0 / 2
phrase: ['адамов' 'веко' 'время']
31400 : 0 / 3
phrase: ['адамов' 'слеза']
31400 : 0 / 2
phrase: ['ад' 'кромешный']
31400 : 0 / 2
phrase: ['ад' 'кромешный' 'на' 'душа' 'сердце' 'у' 'кто']
31400 : 1 / 7
phrase: ['аллилуйя' 'петь' 'кто']
31400 : 0 / 3
phrase: ['алчущие' 'и' 'жаждать']
31400 : 1 / 3
phrase: ['альфа' 'и' 'омега' 'что']
31400 : 1 / 4
phrase: ['аредов' 'веки' 'жить' 'прожить']
31400 : 0 / 4
phrase: ['а' 'судья' 'кто']
31400 : 0 / 3
phrase: ['бальзам' 'на' 'рана' 'для' 'душа' 'сердце' 'лить' 'проливать' 'пролить']
31400 : 1 / 9
phrase: ['башня' 'из' 

In [ ]:
TokenDf.loc[TokenDf.p_id == 31400]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice
0,31400,0,2,На,1_1,1_2,case,ADP,на,None,None,None,None,None,None,None,None,None,None,None
1,31400,3,6,это,1_2,1_3,obl,PRON,это,Inan,None,Acc,None,Neut,None,Sing,None,None,None,None
2,31400,7,13,сказал,1_3,1_0,root,VERB,сказать,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Act
3,31400,14,19,Иисус,1_4,1_3,nsubj,PROPN,иисус,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None
4,31400,21,30,некоторый,1_6,1_7,det,DET,некоторый,None,None,Nom,None,Masc,None,Sing,None,None,None,None
5,31400,31,38,человек,1_7,1_8,nsubj,NOUN,человек,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None
6,31400,39,42,шел,1_8,1_4,parataxis,VERB,идти,None,Imp,None,None,Masc,Ind,Sing,None,Past,Fin,Act
7,31400,43,45,из,1_9,1_10,case,ADP,из,None,None,None,None,None,None,None,None,None,None,None
8,31400,46,56,Иерусалима,1_10,1_8,obl,PROPN,иерусалим,Inan,None,Gen,None,Masc,None,Sing,None,None,None,None
9,31400,57,58,в,1_11,1_12,case,ADP,в,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
FrazyDf.loc[FrazyDf.p_id == 31400].lemma.tolist()

[]

In [ ]:
testPhrase = FrazyDf.loc[FrazyDf.p_id == 22].lemma.unique()

In [ ]:
testVerse = TokenDf.loc[TokenDf.p_id == 31412].lemma.tolist()

In [ ]:
i = 0
print(testPhrase.tolist())
print(testVerse)
for x in testPhrase.tolist():
    if x in testVerse:
        i+=1
print(f"i = {i}")
if i == testPhrase.size:
    print("phrase in verse")

['благой', 'часть', 'избрать']
['а', 'один', 'только', 'нужный', 'мария', 'же', 'избрать', 'благой', 'часть', 'который', 'не', 'отняться', 'у', 'она']
i = 3
phrase in verse


In [ ]:
for x in pos_lists[20:25]:
    print(x)
    for y in TokenDf.p_id:
        #print(TokenDf.loc[TokenDf.p_id == y].lemma.unique())
        #if x in TokenDf.loc[TokenDf.p_id == y].lemma.unique():
            #print(f"{x} in {y}")

['бесплодный' 'смоковница']
['бить' 'колотить' 'ударять' 'себя' 'в' 'грудь']
['благой' 'часть' 'избрать']
['блаженный' 'кто' 'веровать' 'тепло' 'он' 'на' 'свет']
['блаженный' 'муж' 'иже' 'не' 'ида' 'на' 'совет' 'нечестивый']


In [ ]:
FrazyAbbrsDf = BibleFrazyDf.iloc[:105].fraza.str.split(' – ', expand=True).rename(columns={0:"abbr", 1:"fraza"})
FrazyAbbrsDf.loc[:,'abbr'] = FrazyAbbrsDf.loc[:,'abbr'].apply(lambda x: x.strip())
FrazyAbbrsDf.loc[:,'fraza'] = FrazyAbbrsDf.loc[:,'fraza'].apply(lambda x: x.strip())
FrazyAbbrsDict = FrazyAbbrsDf.set_index('abbr').to_dict().get('fraza')
FrazyAbbrsDict

{'АиФ': 'Аргументы и факты',
 'Лит газета': '«Литературная газета»',
 'англ': 'английский',
 'межд': 'междометие',
 'ант': 'антоним антонимы',
 'местн п': 'местный падеж',
 'бел': 'белорусский',
 'МК': '«Московский комсомолец»',
 'библ': 'библейский',
 'болг': 'болгарский',
 'МН': '«Московские новости»',
 'бран': 'бранный',
 'мн ч': 'множественное число',
 'букв': 'буквально',
 'м р': 'мужской род',
 'в': 'век',
 'нареч': 'наречие',
 'вв': 'века',
 'нач': 'начало',
 'возм': 'возможно',
 'НГ': '«Независимая газета»',
 'высок': 'высокий',
 'негативн': 'негативный',
 'г': 'год',
 'неизм': 'неизменяемое',
 'гг': 'годы',
 'нем': 'немецкий',
 'глаг': 'глагол',
 'неодобр': 'неодобрительный',
 'гол': 'голландский',
 'нов': 'новый',
 'греч': 'греческий',
 'Нов изв': '«Новые известия»',
 'грубопрост': 'грубопросторечный',
 'нэ': 'новой эры',
 'дат п': 'дательный падеж',
 'обл': 'областное',
 'диал': 'диалектный',
 'одобр': 'одобрительный',
 'до нэ': 'до новой эры',
 'ок': 'около',
 'дргреч': 'др